In [1]:
import sqlite3
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
# if __name__=='__main__':
    

In [2]:
#Importing DATA
conn = sqlite3.connect('data/raw_data_project_m1.db')

sql_statement='''
          SELECT personal_info.uuid,country_info.country_code, career_info.normalized_job_code, personal_info.age
          FROM personal_info
          JOIN career_info ON personal_info.uuid=career_info.uuid
          JOIN country_info ON personal_info.uuid=country_info.uuid
          JOIN poll_info ON personal_info.uuid=poll_info.uuid
          '''
raw_df=pd.DataFrame(pd.read_sql_query(sql_statement, conn))

raw_df.head()
# raw_df.info()

,uuid,country_code,normalized_job_code,age
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,None,61 years old
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,57 years old
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,None,32 years old
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old


In [40]:
#Cleaning age

#Exploring values
# age_values=raw_df['age'].unique()
# print(list(age_values))

# cleaning
clean_age=[]
for item in raw_df['age']:
    if "years old" in item:
        clean_age.append(int(item[0:2]))
    elif len(item)>=4:
        clean_age.append(2016-int(item))

raw_df['age']=clean_age #add to dataframe

raw_df['age']=raw_df['age']+4 #convert to 2020

TypeError: argument of type 'int' is not iterable

In [75]:
#Cleaning country_code

#Exploring values
# list(raw_df['country_code'].unique())
#Importing csv of country-country code
country_equiv=pd.read_csv('data/country_code.csv',names=['Country','Code'])
#Converting to dictionary
country_dict={}
for i in country_equiv.index:
    k=country_equiv.loc[i,'Code']
    v=country_equiv.loc[i,'Country']
    country_dict[k[1:3]]=v
#Adding missing values
country_dict['GB']='Great Britain'
country_dict['GR']='Greece'
#Adding country to dataframe
for i in raw_df.index:
    code=raw_df.loc[i,'country_code']
    raw_df.loc[i,'country']=country_dict[code]

In [17]:
#REPORTING

In [ ]:
while True:
    try:
        report_type=int(input('''Please enter which type of report you like:
        (1) To get the table for every country included in the dataset 
        (2) To get the table for a specific country imputed by the user'''))
        break
    except:
        pass

In [3]:
url = 'http://api.dataatwork.org/v1/jobs/'
job_names=[]
# for i in raw_df['normalized_job_code']:
#     if i!=None:
#         data=i
#         res = requests.get(url+data)
#         job_title= res.json()
#         job_names.append(job_title['title'])
#     else:
#         job_names.append(None)
   
raw_df['Job Title']=job_names

KeyboardInterrupt: 

In [9]:
url = 'http://api.dataatwork.org/v1/jobs/'
job_dict={}
search_val=list(raw_df['normalized_job_code'].unique())

for val in search_val:
    if val==None:
        job_dict[val]=None
    else:
        data=val
        res = requests.get(url+data)
        job_title= res.json()
        job_dict[val]=job_title['title']
# job_dict
print('Finished!')

{None: None,
 '861a9b9151e11362eb3c77ca914172d0': 'Automatic Data Processing Planner',
 '049a3f3a2b5f85cb2971ba77ad66e10c': 'Data Coordinator',
 'f4b2fb1aa40f661488e2782b6d57ad2f': 'Database Developer',
 '27af8700f5577cec835acee2cb90a2ff': 'Data Entry Specialist',
 'c1b670eba9ccb65e7c99f7da116d5b9c': 'Database Architect',
 'd71c4164c99a5f9bca773e755a72f40f': 'Geographic Information Systems Database Administrator (GIS Database Administrator)',
 '2ee80cf30e2483dfe0947adc0428f8c5': 'Crime Data Specialist',
 '897ef18a9321616495831a4b4b0c1bb2': 'Data Communications Software Consultant',
 'a003702c0cea282e17967772dd83de5e': 'Data Security Analyst',
 '04c82eae29e1c4a30eac0710b74ab276': 'Database Marketing Analyst',
 'fcaf4aac737e53ad24b9f32f08b8740f': 'Computer or Data Processing Systems Consultant',
 '3833bd699e7afe4665b595f282da0387': 'Database Development and Administration Project Manager',
 '19e384e977796c25780d8c28b12089c4': 'Naval Tactical Data System--Combat Information Center Watch O

In [13]:
for i in raw_df.index:
    raw_df.loc[i,'Job Title']= job_dict[raw_df.loc[i,'normalized_job_code']]

In [14]:
raw_df.head()

,uuid,country_code,normalized_job_code,age,Job Title
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,None,61 years old,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,57 years old,Automatic Data Processing Planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,None,32 years old,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old,Data Coordinator
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old,Database Developer
